In [2]:
!pip install dbfread

  Using cached dbfread-2.0.7-py2.py3-none-any.whl (20 kB)


In [3]:
# Lendo banco de dados de São Paulo
import pandas as pd

from dbfread import DBF
filename = 'ESC2019_RMSP_CEM_V1.DBF'
table = DBF(filename, encoding='latin1')    
original = pd.DataFrame(iter(table))
original.to_csv('spCompletona.csv')

original = original[['LONGITUDE1','LATITUDE1']]

original.rename(columns={'LONGITUDE1':'lonGold','LATITUDE1':'latGold' }, inplace = True)



In [ ]:
sp = pd.read_csv('sp.csv')
len(sp) 

In [ ]:
len(original)

In [ ]:
sp_street = sp['street']

# atualize a coluna street do dataframe original com os valores de sp_street
original.update(sp_street)
original.head()

In [ ]:
sp = sp.join(original[['latGold', 'lonGold']])
sp.to_csv('spComp.csv')
sp

In [3]:
import psycopg2

con = psycopg2.connect( host='35.196.180.75',
                            database='SGM',
                            user='postgres', 
                            password='postgres')
cur = con.cursor()

In [ ]:
sp
sp.to_csv('spComp.csv')

In [6]:
table_name = "Response"
table_name2 = "Request"
city_name = "'São Paulo'"
cur.execute('select res.geoapi_id, req.request_id, res.latitude, res.longitude, res.accuracy, req.number, req.district, req.public_place from "Response" as res join "Request" as req on res.request_id = req.request_id where req.request_id >= 9997 and req.request_id <= 22500')
recset = cur.fetchall()



In [7]:
len(recset)

45827

In [8]:
import pandas as pd
geo = pd.DataFrame(recset, columns = ['geoapi_id', 'request_id', 'lat', 'long', 'accuracy', 'number', 'district', 'public_place'])

In [9]:
#geo = geo[geo['request_id'] >= 9997]
geo.head()

,geoapi_id,request_id,lat,long,accuracy,number,district,public_place
0,OpenRouteService,10420,-22.154376,-48.428808,0.3,50,ITAPECERICA DA SERRA,RUA NICOLA FELICE
1,OpenRouteService,10422,-22.154376,-48.428808,0.3,103,ITAPECERICA DA SERRA,RUA ANTONIO DE ALBUQUERQUE
2,OpenRouteService,10426,-22.154376,-48.428808,0.3,101,ITAPECERICA DA SERRA,RUA MARIA APARECIDA PEIXOTO MARINHO BARBOSA
3,OpenRouteService,10428,-22.154376,-48.428808,0.3,21,ITAPECERICA DA SERRA,RUA ASIA
4,OpenRouteService,10431,-22.154376,-48.428808,0.3,400,ITAPECERICA DA SERRA,RUA TERRA NOVA


In [10]:
geo = geo.sort_values(by='request_id')
geo = geo.reset_index(drop=True)
geo.head()

,geoapi_id,request_id,lat,long,accuracy,number,district,public_place
0,Google,9997,-23.986137,-47.144314,NaN,285,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
1,TomTom,9997,-23.985790,-47.144410,14.062144,285,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
2,MapBox,9997,-22.555770,-47.401660,0.638148,285,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
3,OpenRouteService,9997,-23.938906,-47.024205,0.600000,285,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
4,TomTom,9998,-23.174240,-46.570320,8.543887,09,JUQUITIBA,ESTRADA DAS PEREIRAS


In [11]:
geo.to_csv('pesqSaoPaulo.csv')

In [ ]:
1243587

In [1]:
import psycopg2

con = psycopg2.connect( host='34.23.173.147',
                            database='SGM',
                            user='terralab', 
                            password='terralab0705')
cur = con.cursor()


In [ ]:
cur.execute('select geoapi_id, request_id, latitude, longitude, accuracy, number, district, public_place from "Response" where request_id >= 1243587')
#cur.execute('select request_id from "Request" where request_id >= 1243587')
recset = cur.fetchall()

In [ ]:
len(recset)

In [6]:
def get_max_request_id(cur):
    
    query = """ SELECT *
                FROM "Response"
                WHERE geoapi_id = 'Google'
                ORDER BY request_id DESC
                LIMIT 1
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior

ultimo = get_max_request_id(cur)

In [3]:
ultimo

[(2852599,
  'RUA PADRE ARGEMIRO MOREIRA, 1300, BELO HORIZONTE, MG',
  'RUA PADRE ARGEMIRO MOREIRA',
  '',
  'BELO HORIZONTE',
  'MG',
  '31998250',
  None,
  '1300')]

In [ ]:
import pandas as pd
geo = pd.DataFrame(recset, columns = ['geoapi_id', 'request_id', 'lat', 'long', 'accuracy', 'number', 'district', 'public_place'])

In [ ]:
geo = geo.sort_values(by='request_id')
geo = geo.reset_index(drop=True)
geo.head()

In [ ]:
geo.to_csv('respostaBh.csv')

In [18]:
import pandas as pd

In [12]:
original = pd.read_csv('spComp.csv')
original.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,number,street,city,district,postcode,latGold,lonGold
0,0,0,0,285,RUA ANTONIO FRANCISCO PEREIRA,JUQUITIBA,BARNABES,NaN,-23.986886,-47.143986
1,1,1,1,09,ESTRADA DAS PEREIRAS,JUQUITIBA,JUQUITIBA,NaN,-23.992136,-47.090648
2,2,2,2,31,RUA ANTONIO SOARES GODINHO,JUQUITIBA,BARNABES,NaN,-23.985717,-47.145545
3,3,3,3,35,RUA ANTONIO SOARES GODINHO,JUQUITIBA,BARNABES,NaN,-23.985685,-47.145560
4,4,4,4,413,RUA ANTONIO SOARES GODINHO,JUQUITIBA,BARNABES,NaN,-23.983873,-47.146239


In [4]:
geo = pd.read_csv('pesqSaoPaulo.csv')

In [13]:
#merged_df = pd.merge(geo, original, how='inner', left_on='public_place', right_on='street')
#result_df = merged_df[merged_df['number_x'] == merged_df['number_y']]

# Renomeia as colunas
#result_df = result_df[['public_place', 'number_x']].rename(columns={'number_x': 'number'})

result_df = original.merge(geo, how='inner', left_on=['street', 'number'], right_on=['public_place', 'number'])
result_df.head(50)

#merged.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,number,street,city,district_x,postcode,latGold,lonGold,geoapi_id,request_id,lat,long,accuracy,district_y,public_place
0,0,0,0,285,RUA ANTONIO FRANCISCO PEREIRA,JUQUITIBA,BARNABES,NaN,-23.986886,-47.143986,Google,9997,-23.986137,-47.144314,NaN,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
1,0,0,0,285,RUA ANTONIO FRANCISCO PEREIRA,JUQUITIBA,BARNABES,NaN,-23.986886,-47.143986,TomTom,9997,-23.985790,-47.144410,14.062144,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
2,0,0,0,285,RUA ANTONIO FRANCISCO PEREIRA,JUQUITIBA,BARNABES,NaN,-23.986886,-47.143986,MapBox,9997,-22.555770,-47.401660,0.638148,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
3,0,0,0,285,RUA ANTONIO FRANCISCO PEREIRA,JUQUITIBA,BARNABES,NaN,-23.986886,-47.143986,OpenRouteService,9997,-23.938906,-47.024205,0.600000,BARNABES,RUA ANTONIO FRANCISCO PEREIRA
4,1,1,1,09,ESTRADA DAS PEREIRAS,JUQUITIBA,JUQUITIBA,NaN,-23.992136,-47.090648,TomTom,9998,-23.174240,-46.570320,8.543887,JUQUITIBA,ESTRADA DAS PEREIRAS
5,1,1,1,09,ESTRADA DAS PEREIRAS,JUQUITIBA,JUQUITIBA,NaN,-23.992136,-47.090648,MapBox,9998,-23.164259,-46.569506,0.667850,JUQUITIBA,ESTRADA DAS PEREIRAS
6,1,1,1,09,ESTRADA DAS PEREIRAS,JUQUITIBA,JUQUITIBA,NaN,-23.992136,-47.090648,OpenRouteService,9998,-23.938906,-47.024205,0.600000,JUQUITIBA,ESTRADA DAS PEREIRAS
7,1,1,1,09,ESTRADA DAS PEREIRAS,JUQUITIBA,JUQUITIBA,NaN,-23.992136,-47.090648,Google,9998,-23.836693,-47.040083,NaN,JUQUITIBA,ESTRADA DAS PEREIRAS
8,2,2,2,31,RUA ANTONIO SOARES GODINHO,JUQUITIBA,BARNABES,NaN,-23.985717,-47.145545,Google,9999,-23.985665,-47.145556,NaN,BARNABES,RUA ANTONIO SOARES GODINHO
9,2,2,2,31,RUA ANTONIO SOARES GODINHO,JUQUITIBA,BARNABES,NaN,-23.985717,-47.145545,OpenRouteService,9999,-23.938906,-47.024205,0.600000,BARNABES,RUA ANTONIO SOARES GODINHO


In [18]:
aux = result_df.drop_duplicates(subset=['request_id', 'geoapi_id'])

In [19]:
len(result_df)

48024

In [22]:
result_df = aux

In [23]:
result_df.to_csv('DadosGeoeGold.csv')

In [24]:
geo['geoapi_id'].unique().tolist()

['Google', 'TomTom', 'MapBox', 'OpenRouteService']